In [0]:
import os, logging, json
from datetime import datetime
import tempfile

LOG_DIR_PREFERENCE = [  # in order
    os.getenv("SECURE_LOG_DIR"),             # let you override if you want
    "/databricks/driver/logs",               # usually writable on CE
    "/local_disk0/logs",                     # some runtimes expose this
    None,                                    # will use tempfile.gettempdir()
]

class JsonFormatter(logging.Formatter):
    def format(self, record):
        data = {
            "ts": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "level": record.levelname,
            "name": record.name,
            "message": record.getMessage(),
            "module": record.module,
            "funcName": record.funcName,
            "lineno": record.lineno,
        }
        if record.exc_info:
            data["exc_info"] = self.formatException(record.exc_info)
        return json.dumps(data)

def _pick_log_dir(preferred: str | None = None) -> str | None:
    """Return a writable directory or None (console only)."""
    candidates = [preferred] + LOG_DIR_PREFERENCE if preferred is not None else LOG_DIR_PREFERENCE
    for cand in candidates:
        try:
            if cand is None:
                cand = os.path.join(tempfile.gettempdir(), "logs")
            os.makedirs(cand, exist_ok=True)
            # probe write permission
            test_path = os.path.join(cand, ".probe")
            with open(test_path, "w") as f:
                f.write("ok")
            os.remove(test_path)
            return cand
        except Exception:
            continue
    return None  # no writable dir; skip file handler

def reset_logger(name="app"):
    logger = logging.getLogger(name)
    for h in list(logger.handlers):
        try: h.close()
        except Exception: pass
        logger.removeHandler(h)
    if hasattr(logger, "_configured"):
        delattr(logger, "_configured")

def get_logger(name="app", log_dir: str | None = None):
    """Console logger always; file logging added only if a writable dir is found."""
    logger = logging.getLogger(name)
    if getattr(logger, "_configured", False):
        return logger

    logger.setLevel(logging.INFO)

    # console
    ch = logging.StreamHandler()
    ch.setFormatter(JsonFormatter())
    logger.addHandler(ch)

    # file (best-effort)
    chosen_dir = _pick_log_dir(log_dir)
    if chosen_dir:
        try:
            log_file = os.path.join(chosen_dir, f"run_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log")
            fh = logging.FileHandler(log_file)
            fh.setFormatter(JsonFormatter())
            logger.addHandler(fh)
            logger.info(json.dumps({
                "ts": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "level": "INFO",
                "name": name,
                "message": f"File logging enabled at {log_file}",
            }))
        except Exception as e:
            logger.warning(json.dumps({
                "ts": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                "level": "WARNING",
                "name": name,
                "message": f"File logging disabled: {type(e).__name__}: {e}",
            }))
    else:
        logger.info(json.dumps({
            "ts": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            "level": "INFO",
            "name": name,
            "message": "File logging disabled (no writable directory found); console only.",
        }))

    logger._configured = True
    return logger
